In [1]:
import pandas as pd 
import numpy as np

In [2]:
dataset = pd.read_csv('ratings.csv')
dataset.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
movieTitles = pd.read_csv('movies.csv')
movieTitles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
dataset = pd.merge(dataset, movieTitles, on='movieId')
dataset.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [5]:
movieMatrix = dataset.pivot_table(index='userId', columns='movieId', values='rating')
movieMatrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
R = movieMatrix.transpose().fillna(movieMatrix.transpose().mean())
R = R.transpose()
R.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,4.366379,4.000000,4.366379,4.366379,4.000000,4.366379,4.366379,4.366379,4.366379,...,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379,4.366379
2,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,...,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276,3.948276
3,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,...,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897,2.435897
4,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,...,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556,3.555556
5,4.000000,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,...,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364,3.636364


In [7]:
R = np.array(R)
R.shape

(610, 9724)

In [8]:
numUsers, numItems = R.shape

In [9]:
np.linalg.matrix_rank(R)

610

In [21]:
K = 400

In [22]:
U = np.random.randn(numUsers, K)
U

array([[-0.40570908, -0.2969521 ,  1.01983176, ...,  0.31115748,
         0.37419758,  0.68798733],
       [ 0.45902229,  0.46958031,  1.50179602, ...,  0.16241656,
        -1.5376978 ,  0.26664675],
       [-0.43649914,  0.1610729 , -0.26839717, ..., -0.3986888 ,
        -2.3102413 , -0.27612477],
       ...,
       [ 0.17368305,  1.57727778,  0.46322642, ..., -1.13852563,
         0.99197921,  1.0645807 ],
       [ 0.51982596, -0.98388572,  1.1383051 , ...,  0.31651445,
        -0.91889431,  0.71658648],
       [ 1.11534309, -0.21101477,  0.69930883, ...,  0.90396593,
        -0.93373504, -1.96405278]])

In [23]:
V = np.random.randn(numItems, K)
V

array([[0.40009162, 0.79700308, 0.79797395, ..., 0.91415489, 0.98061291,
        0.10488484],
       [0.72460728, 0.44299458, 0.2566177 , ..., 0.26840639, 0.32696232,
        0.78545472],
       [0.39852159, 0.59669885, 0.67007099, ..., 0.27742997, 0.22470526,
        0.54715947],
       ...,
       [0.70018508, 0.56944917, 0.40018184, ..., 0.01209188, 0.34129428,
        0.61593983],
       [0.36985047, 0.66076135, 0.61105155, ..., 0.8774237 , 0.55649409,
        0.87024413],
       [0.92383194, 0.72082627, 0.55609846, ..., 0.94202146, 0.88123041,
        0.19371782]])

In [13]:
where_are_NaNs = np.isnan(movieMatrix)
where_are_NaNs.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,False,True,False,True,True,False,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,False,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [24]:
alpha = 0.000001

In [25]:
for i in range(20000):
    error = R - np.matmul(U, V.transpose())
    
    #Batch Update
    U = U + alpha * np.matmul(error, V)
    V = V + alpha * np.matmul(error.transpose(), U)
    
    # Calculating norm
    error[where_are_NaNs] = 0
    print("Iteration: ", i, " ---> Frobenius norm: ", np.linalg.norm(error))

Iteration:  0  ---> Frobenius norm:  4082.367786017278
Iteration:  1  ---> Frobenius norm:  1848.6585184210962
Iteration:  2  ---> Frobenius norm:  1842.1253741991757
Iteration:  3  ---> Frobenius norm:  1838.793917105875
Iteration:  4  ---> Frobenius norm:  1835.4922382019777
Iteration:  5  ---> Frobenius norm:  1832.2039516380212
Iteration:  6  ---> Frobenius norm:  1828.9285825920451
Iteration:  7  ---> Frobenius norm:  1825.6660383813871
Iteration:  8  ---> Frobenius norm:  1822.4162373982836
Iteration:  9  ---> Frobenius norm:  1819.1790990364925
Iteration:  10  ---> Frobenius norm:  1815.954543409013
Iteration:  11  ---> Frobenius norm:  1812.7424913325463
Iteration:  12  ---> Frobenius norm:  1809.5428643193263
Iteration:  13  ---> Frobenius norm:  1806.3555845692633
Iteration:  14  ---> Frobenius norm:  1803.1805749621979
Iteration:  15  ---> Frobenius norm:  1800.0177590502633
Iteration:  16  ---> Frobenius norm:  1796.8670610503539
Iteration:  17  ---> Frobenius norm:  1793.7

Iteration:  144  ---> Frobenius norm:  1472.5168131990806
Iteration:  145  ---> Frobenius norm:  1470.470889628621
Iteration:  146  ---> Frobenius norm:  1468.4309732502768
Iteration:  147  ---> Frobenius norm:  1466.3970358532306
Iteration:  148  ---> Frobenius norm:  1464.3690494112634
Iteration:  149  ---> Frobenius norm:  1462.3469860812038
Iteration:  150  ---> Frobenius norm:  1460.3308182013961
Iteration:  151  ---> Frobenius norm:  1458.3205182901834
Iteration:  152  ---> Frobenius norm:  1456.316059044404
Iteration:  153  ---> Frobenius norm:  1454.317413337906
Iteration:  154  ---> Frobenius norm:  1452.3245542200743
Iteration:  155  ---> Frobenius norm:  1450.3374549143766
Iteration:  156  ---> Frobenius norm:  1448.3560888169193
Iteration:  157  ---> Frobenius norm:  1446.3804294950212
Iteration:  158  ---> Frobenius norm:  1444.4104506858002
Iteration:  159  ---> Frobenius norm:  1442.4461262947755
Iteration:  160  ---> Frobenius norm:  1440.487430394482
Iteration:  161  -

Iteration:  287  ---> Frobenius norm:  1229.979758457482
Iteration:  288  ---> Frobenius norm:  1228.5744162926137
Iteration:  289  ---> Frobenius norm:  1227.1723799630913
Iteration:  290  ---> Frobenius norm:  1225.7736372038748
Iteration:  291  ---> Frobenius norm:  1224.3781758133591
Iteration:  292  ---> Frobenius norm:  1222.9859836529517
Iteration:  293  ---> Frobenius norm:  1221.597048646657
Iteration:  294  ---> Frobenius norm:  1220.211358780663
Iteration:  295  ---> Frobenius norm:  1218.8289021029304
Iteration:  296  ---> Frobenius norm:  1217.4496667227875
Iteration:  297  ---> Frobenius norm:  1216.0736408105251
Iteration:  298  ---> Frobenius norm:  1214.7008125969976
Iteration:  299  ---> Frobenius norm:  1213.3311703732247
Iteration:  300  ---> Frobenius norm:  1211.964702489999
Iteration:  301  ---> Frobenius norm:  1210.601397357494
Iteration:  302  ---> Frobenius norm:  1209.2412434448781
Iteration:  303  ---> Frobenius norm:  1207.8842292799281
Iteration:  304  --

Iteration:  429  ---> Frobenius norm:  1058.6567796619834
Iteration:  430  ---> Frobenius norm:  1057.6221434827796
Iteration:  431  ---> Frobenius norm:  1056.5895537934953
Iteration:  432  ---> Frobenius norm:  1055.559004314805
Iteration:  433  ---> Frobenius norm:  1054.5304887942896
Iteration:  434  ---> Frobenius norm:  1053.5040010062887
Iteration:  435  ---> Frobenius norm:  1052.479534751755
Iteration:  436  ---> Frobenius norm:  1051.4570838581076
Iteration:  437  ---> Frobenius norm:  1050.4366421790874
Iteration:  438  ---> Frobenius norm:  1049.4182035946146
Iteration:  439  ---> Frobenius norm:  1048.4017620106445
Iteration:  440  ---> Frobenius norm:  1047.3873113590262
Iteration:  441  ---> Frobenius norm:  1046.3748455973628
Iteration:  442  ---> Frobenius norm:  1045.3643587088693
Iteration:  443  ---> Frobenius norm:  1044.3558447022351
Iteration:  444  ---> Frobenius norm:  1043.349297611487
Iteration:  445  ---> Frobenius norm:  1042.3447114958494
Iteration:  446  

Iteration:  718  ---> Frobenius norm:  825.652955800261
Iteration:  719  ---> Frobenius norm:  825.0251814772874
Iteration:  720  ---> Frobenius norm:  824.3983638499246
Iteration:  721  ---> Frobenius norm:  823.772500752234
Iteration:  722  ---> Frobenius norm:  823.1475900251468
Iteration:  723  ---> Frobenius norm:  822.5236295164364
Iteration:  724  ---> Frobenius norm:  821.9006170806904
Iteration:  725  ---> Frobenius norm:  821.2785505792826
Iteration:  726  ---> Frobenius norm:  820.6574278803463
Iteration:  727  ---> Frobenius norm:  820.0372468587449
Iteration:  728  ---> Frobenius norm:  819.4180053960473
Iteration:  729  ---> Frobenius norm:  818.7997013804982
Iteration:  730  ---> Frobenius norm:  818.1823327069925
Iteration:  731  ---> Frobenius norm:  817.5658972770483
Iteration:  732  ---> Frobenius norm:  816.9503929987791
Iteration:  733  ---> Frobenius norm:  816.3358177868685
Iteration:  734  ---> Frobenius norm:  815.7221695625425
Iteration:  735  ---> Frobenius n

...

Iteration:  19397  ---> Frobenius norm:  226.91038115012194
Iteration:  19398  ---> Frobenius norm:  226.90996460015268
Iteration:  19399  ---> Frobenius norm:  226.9095480714595
Iteration:  19400  ---> Frobenius norm:  226.909131564036
Iteration:  19401  ---> Frobenius norm:  226.90871507787566
Iteration:  19402  ---> Frobenius norm:  226.90829861297206
Iteration:  19403  ---> Frobenius norm:  226.90788216931873
Iteration:  19404  ---> Frobenius norm:  226.9074657469092
Iteration:  19405  ---> Frobenius norm:  226.90704934573697
Iteration:  19406  ---> Frobenius norm:  226.9066329657957
Iteration:  19407  ---> Frobenius norm:  226.9062166070788
Iteration:  19408  ---> Frobenius norm:  226.90580026957983
Iteration:  19409  ---> Frobenius norm:  226.90538395329247
Iteration:  19410  ---> Frobenius norm:  226.9049676582101
Iteration:  19411  ---> Frobenius norm:  226.90455138432637
Iteration:  19412  ---> Frobenius norm:  226.9041351316348
Iteration:  19413  ---> Frobenius norm:  226.903

Iteration:  19535  ---> Frobenius norm:  226.85309563160496
Iteration:  19536  ---> Frobenius norm:  226.85268195736577
Iteration:  19537  ---> Frobenius norm:  226.85226830352715
Iteration:  19538  ---> Frobenius norm:  226.85185467008276
Iteration:  19539  ---> Frobenius norm:  226.85144105702642
Iteration:  19540  ---> Frobenius norm:  226.8510274643519
Iteration:  19541  ---> Frobenius norm:  226.85061389205302
Iteration:  19542  ---> Frobenius norm:  226.85020034012348
Iteration:  19543  ---> Frobenius norm:  226.84978680855716
Iteration:  19544  ---> Frobenius norm:  226.8493732973478
Iteration:  19545  ---> Frobenius norm:  226.8489598064892
Iteration:  19546  ---> Frobenius norm:  226.84854633597516
Iteration:  19547  ---> Frobenius norm:  226.8481328857995
Iteration:  19548  ---> Frobenius norm:  226.84771945595594
Iteration:  19549  ---> Frobenius norm:  226.84730604643832
Iteration:  19550  ---> Frobenius norm:  226.8468926572405
Iteration:  19551  ---> Frobenius norm:  226.

Iteration:  19673  ---> Frobenius norm:  226.7961987915673
Iteration:  19674  ---> Frobenius norm:  226.79578787453087
Iteration:  19675  ---> Frobenius norm:  226.7953769770527
Iteration:  19676  ---> Frobenius norm:  226.79496609912698
Iteration:  19677  ---> Frobenius norm:  226.79455524074754
Iteration:  19678  ---> Frobenius norm:  226.7941444019085
Iteration:  19679  ---> Frobenius norm:  226.79373358260386
Iteration:  19680  ---> Frobenius norm:  226.79332278282766
Iteration:  19681  ---> Frobenius norm:  226.79291200257393
Iteration:  19682  ---> Frobenius norm:  226.79250124183667
Iteration:  19683  ---> Frobenius norm:  226.79209050060985
Iteration:  19684  ---> Frobenius norm:  226.79167977888767
Iteration:  19685  ---> Frobenius norm:  226.79126907666398
Iteration:  19686  ---> Frobenius norm:  226.79085839393287
Iteration:  19687  ---> Frobenius norm:  226.7904477306884
Iteration:  19688  ---> Frobenius norm:  226.79003708692468
Iteration:  19689  ---> Frobenius norm:  226

Iteration:  19811  ---> Frobenius norm:  226.73967458515688
Iteration:  19812  ---> Frobenius norm:  226.7392663113154
Iteration:  19813  ---> Frobenius norm:  226.7388580562221
Iteration:  19814  ---> Frobenius norm:  226.73844981987125
Iteration:  19815  ---> Frobenius norm:  226.73804160225708
Iteration:  19816  ---> Frobenius norm:  226.73763340337382
Iteration:  19817  ---> Frobenius norm:  226.73722522321575
Iteration:  19818  ---> Frobenius norm:  226.7368170617771
Iteration:  19819  ---> Frobenius norm:  226.7364089190521
Iteration:  19820  ---> Frobenius norm:  226.73600079503504
Iteration:  19821  ---> Frobenius norm:  226.73559268972014
Iteration:  19822  ---> Frobenius norm:  226.73518460310171
Iteration:  19823  ---> Frobenius norm:  226.73477653517395
Iteration:  19824  ---> Frobenius norm:  226.73436848593113
Iteration:  19825  ---> Frobenius norm:  226.73396045536754
Iteration:  19826  ---> Frobenius norm:  226.73355244347744
Iteration:  19827  ---> Frobenius norm:  226

Iteration:  19949  ---> Frobenius norm:  226.68350757757142
Iteration:  19950  ---> Frobenius norm:  226.68310183723898
Iteration:  19951  ---> Frobenius norm:  226.68269611487526
Iteration:  19952  ---> Frobenius norm:  226.68229041047468
Iteration:  19953  ---> Frobenius norm:  226.68188472403165
Iteration:  19954  ---> Frobenius norm:  226.68147905554073
Iteration:  19955  ---> Frobenius norm:  226.6810734049963
Iteration:  19956  ---> Frobenius norm:  226.68066777239287
Iteration:  19957  ---> Frobenius norm:  226.68026215772488
Iteration:  19958  ---> Frobenius norm:  226.67985656098682
Iteration:  19959  ---> Frobenius norm:  226.6794509821732
Iteration:  19960  ---> Frobenius norm:  226.6790454212784
Iteration:  19961  ---> Frobenius norm:  226.67863987829693
Iteration:  19962  ---> Frobenius norm:  226.67823435322327
Iteration:  19963  ---> Frobenius norm:  226.67782884605197
Iteration:  19964  ---> Frobenius norm:  226.67742335677738
Iteration:  19965  ---> Frobenius norm:  22

In [27]:
finalFilledMatrix = np.matmul(U, V.transpose())
finalFilledMatrix

array([[4.22940432, 4.63898662, 4.23252202, ..., 4.32498695, 4.34961054,
        4.32206054],
       [4.40041123, 3.84866142, 3.89805687, ..., 4.04056668, 3.89626564,
        3.90037544],
       [2.59187277, 2.64675126, 2.22989801, ..., 2.33121321, 2.50864577,
        2.3875869 ],
       ...,
       [2.71838877, 2.80179495, 2.72048833, ..., 3.21981737, 3.12527405,
        3.05071318],
       [3.11752382, 3.17220518, 3.400793  , ..., 3.25847635, 3.28151027,
        3.19550587],
       [4.10968184, 3.47250134, 3.66892405, ..., 3.66208635, 3.62905322,
        3.71548478]])

In [37]:
errorBatch = R-finalFilledMatrix
errorBatch[where_are_NaNs] = 0
np.linalg.norm(errorBatch)

226.6628376156856

## Regularization

In [44]:
UReg =  np.random.randn(numUsers, K)
UReg

array([[-0.75510537, -0.92086619, -0.61819536, ..., -0.55199764,
         0.5190747 , -2.05905411],
       [ 1.63016048, -0.49974131, -0.91334021, ...,  0.32772998,
         0.36341767, -0.8548127 ],
       [-0.25287913, -0.02512365, -1.71444862, ...,  0.97422736,
         1.857965  ,  0.75180621],
       ...,
       [ 2.21780232,  0.57969785, -1.24164887, ...,  1.18317987,
        -2.01697223, -0.07338759],
       [ 0.34376428,  0.72150183, -0.32789761, ...,  0.3979564 ,
        -1.44570065, -0.11498594],
       [ 0.0631124 , -0.26601897,  1.26063998, ..., -0.09968713,
         1.18050326, -0.80369928]])

In [45]:
VReg = np.random.randn(numItems, K)
VReg

array([[ 0.43910634,  1.27729781,  1.75844176, ...,  0.43022947,
         0.54242412,  0.98619169],
       [ 0.30819051,  0.73378606,  0.33392826, ...,  0.32045536,
         0.38556663, -2.10066668],
       [ 1.04290052,  0.59197851, -0.05665875, ...,  0.43202401,
        -0.93886268,  0.40800758],
       ...,
       [-1.80920496, -2.50593774,  0.78603613, ...,  0.05059978,
        -0.45251606,  0.92641594],
       [-0.32641289, -0.23297568, -0.82346866, ..., -0.32551043,
        -0.45633151,  0.92897996],
       [ 0.3118331 ,  0.06233922, -0.74952356, ...,  1.71998204,
        -1.13650395,  0.19656436]])

In [46]:
regularizationFactor = 100

In [47]:
for i in range(20000):
    error = R - np.matmul(UReg, VReg.transpose())
    
    #Batch Update
    UReg = (1 - alpha*regularizationFactor) * UReg + alpha * np.matmul(error, VReg)
    VReg = VReg + alpha * np.matmul(error.transpose(), UReg)
    
    # Calculating norm
    error[where_are_NaNs] = 0
    print("Iteration: ", i, " ---> Frobenius norm: ", np.linalg.norm(error))

Iteration:  0  ---> Frobenius norm:  6469.352527957078
Iteration:  1  ---> Frobenius norm:  6398.972968093187
Iteration:  2  ---> Frobenius norm:  6329.704916862722
Iteration:  3  ---> Frobenius norm:  6261.522897060181
Iteration:  4  ---> Frobenius norm:  6194.402283730314
Iteration:  5  ---> Frobenius norm:  6128.319266838122
Iteration:  6  ---> Frobenius norm:  6063.25081592605
Iteration:  7  ---> Frobenius norm:  5999.174646634341
Iteration:  8  ---> Frobenius norm:  5936.069188969461
Iteration:  9  ---> Frobenius norm:  5873.913557213604
Iteration:  10  ---> Frobenius norm:  5812.687521375811
Iteration:  11  ---> Frobenius norm:  5752.371480092128
Iteration:  12  ---> Frobenius norm:  5692.9464348885795
Iteration:  13  ---> Frobenius norm:  5634.393965726657
Iteration:  14  ---> Frobenius norm:  5576.696207756372
Iteration:  15  ---> Frobenius norm:  5519.835829207007
Iteration:  16  ---> Frobenius norm:  5463.796010350269
Iteration:  17  ---> Frobenius norm:  5408.560423474888
It

Iteration:  145  ---> Frobenius norm:  1951.9299181825124
Iteration:  146  ---> Frobenius norm:  1940.4179704185494
Iteration:  147  ---> Frobenius norm:  1929.0340868233652
Iteration:  148  ---> Frobenius norm:  1917.776698128527
Iteration:  149  ---> Frobenius norm:  1906.6442518533274
Iteration:  150  ---> Frobenius norm:  1895.6352120456652
Iteration:  151  ---> Frobenius norm:  1884.7480590296404
Iteration:  152  ---> Frobenius norm:  1873.9812891596966
Iteration:  153  ---> Frobenius norm:  1863.3334145811568
Iteration:  154  ---> Frobenius norm:  1852.8029629969853
Iteration:  155  ---> Frobenius norm:  1842.3884774406338
Iteration:  156  ---> Frobenius norm:  1832.0885160548069
Iteration:  157  ---> Frobenius norm:  1821.9016518760186
Iteration:  158  ---> Frobenius norm:  1811.8264726247794
Iteration:  159  ---> Frobenius norm:  1801.8615805012857
Iteration:  160  ---> Frobenius norm:  1792.0055919864674
Iteration:  161  ---> Frobenius norm:  1782.2571376482622
Iteration:  162

Iteration:  287  ---> Frobenius norm:  1067.5552182195643
Iteration:  288  ---> Frobenius norm:  1063.9933756348903
Iteration:  289  ---> Frobenius norm:  1060.444364222321
Iteration:  290  ---> Frobenius norm:  1056.9079196616162
Iteration:  291  ---> Frobenius norm:  1053.3837833000273
Iteration:  292  ---> Frobenius norm:  1049.8717021127272
Iteration:  293  ---> Frobenius norm:  1046.3714286629659
Iteration:  294  ---> Frobenius norm:  1042.8827210619445
Iteration:  295  ---> Frobenius norm:  1039.4053429284036
Iteration:  296  ---> Frobenius norm:  1035.9390633479184
Iteration:  297  ---> Frobenius norm:  1032.4836568318985
Iteration:  298  ---> Frobenius norm:  1029.038903276284
Iteration:  299  ---> Frobenius norm:  1025.6045879199357
Iteration:  300  ---> Frobenius norm:  1022.180501302715
Iteration:  301  ---> Frobenius norm:  1018.7664392232457
Iteration:  302  ---> Frobenius norm:  1015.3622026963603
Iteration:  303  ---> Frobenius norm:  1011.9675979102203
Iteration:  304  

Iteration:  431  ---> Frobenius norm:  633.0306856411291
Iteration:  432  ---> Frobenius norm:  630.545405783869
Iteration:  433  ---> Frobenius norm:  628.0697690501997
Iteration:  434  ---> Frobenius norm:  625.6038283299251
Iteration:  435  ---> Frobenius norm:  623.1476356949884
Iteration:  436  ---> Frobenius norm:  620.7012423828854
Iteration:  437  ---> Frobenius norm:  618.2646987807617
Iteration:  438  ---> Frobenius norm:  615.8380544101784
Iteration:  439  ---> Frobenius norm:  613.4213579125477
Iteration:  440  ---> Frobenius norm:  611.0146570352293
Iteration:  441  ---> Frobenius norm:  608.6179986182807
Iteration:  442  ---> Frobenius norm:  606.2314285818547
Iteration:  443  ---> Frobenius norm:  603.8549919142388
Iteration:  444  ---> Frobenius norm:  601.4887326605259
Iteration:  445  ---> Frobenius norm:  599.1326939119115
Iteration:  446  ---> Frobenius norm:  596.7869177956092
Iteration:  447  ---> Frobenius norm:  594.4514454653735
Iteration:  448  ---> Frobenius 

Iteration:  575  ---> Frobenius norm:  383.93043276580966
Iteration:  576  ---> Frobenius norm:  382.92831057057293
Iteration:  577  ---> Frobenius norm:  381.93457979904963
Iteration:  578  ---> Frobenius norm:  380.9491933151165
Iteration:  579  ---> Frobenius norm:  379.97210387679894
Iteration:  580  ---> Frobenius norm:  379.00326414328197
Iteration:  581  ---> Frobenius norm:  378.04262668183844
Iteration:  582  ---> Frobenius norm:  377.0901439746726
Iteration:  583  ---> Frobenius norm:  376.14576842568096
Iteration:  584  ---> Frobenius norm:  375.20945236712976
Iteration:  585  ---> Frobenius norm:  374.2811480662488
Iteration:  586  ---> Frobenius norm:  373.3608077317421
Iteration:  587  ---> Frobenius norm:  372.44838352021543
Iteration:  588  ---> Frobenius norm:  371.54382754252026
Iteration:  589  ---> Frobenius norm:  370.6470918700157
Iteration:  590  ---> Frobenius norm:  369.7581285407464
Iteration:  591  ---> Frobenius norm:  368.87688956553876
Iteration:  592  ---

...

Iteration:  1711  ---> Frobenius norm:  281.0723722223301
Iteration:  1712  ---> Frobenius norm:  281.07236540283026
Iteration:  1713  ---> Frobenius norm:  281.0723586295611
Iteration:  1714  ---> Frobenius norm:  281.07235190203005
Iteration:  1715  ---> Frobenius norm:  281.07234521974976
Iteration:  1716  ---> Frobenius norm:  281.07233858223833
Iteration:  1717  ---> Frobenius norm:  281.07233198901883
Iteration:  1718  ---> Frobenius norm:  281.0723254396197
Iteration:  1719  ---> Frobenius norm:  281.0723189335744
Iteration:  1720  ---> Frobenius norm:  281.07231247042137
Iteration:  1721  ---> Frobenius norm:  281.0723060497042
Iteration:  1722  ---> Frobenius norm:  281.0722996709713
Iteration:  1723  ---> Frobenius norm:  281.0722933337759
Iteration:  1724  ---> Frobenius norm:  281.0722870376763
Iteration:  1725  ---> Frobenius norm:  281.0722807822353
Iteration:  1726  ---> Frobenius norm:  281.07227456702066
Iteration:  1727  ---> Frobenius norm:  281.0722683916046
Iterati

Iteration:  1852  ---> Frobenius norm:  281.07170578888696
Iteration:  1853  ---> Frobenius norm:  281.0717023652987
Iteration:  1854  ---> Frobenius norm:  281.07169895215173
Iteration:  1855  ---> Frobenius norm:  281.07169554933876
Iteration:  1856  ---> Frobenius norm:  281.07169215675333
Iteration:  1857  ---> Frobenius norm:  281.07168877429035
Iteration:  1858  ---> Frobenius norm:  281.0716854018457
Iteration:  1859  ---> Frobenius norm:  281.0716820393164
Iteration:  1860  ---> Frobenius norm:  281.07167868660054
Iteration:  1861  ---> Frobenius norm:  281.0716753435974
Iteration:  1862  ---> Frobenius norm:  281.0716720102072
Iteration:  1863  ---> Frobenius norm:  281.07166868633124
Iteration:  1864  ---> Frobenius norm:  281.07166537187203
Iteration:  1865  ---> Frobenius norm:  281.071662066733
Iteration:  1866  ---> Frobenius norm:  281.07165877081843
Iteration:  1867  ---> Frobenius norm:  281.07165548403395
Iteration:  1868  ---> Frobenius norm:  281.071652206286
Iterat

Iteration:  1993  ---> Frobenius norm:  281.0712907632495
Iteration:  1994  ---> Frobenius norm:  281.07128812569675
Iteration:  1995  ---> Frobenius norm:  281.0712854906901
Iteration:  1996  ---> Frobenius norm:  281.0712828582051
Iteration:  1997  ---> Frobenius norm:  281.0712802282179
Iteration:  1998  ---> Frobenius norm:  281.07127760070455
Iteration:  1999  ---> Frobenius norm:  281.0712749756418
Iteration:  2000  ---> Frobenius norm:  281.0712723530063
Iteration:  2001  ---> Frobenius norm:  281.071269732775
Iteration:  2002  ---> Frobenius norm:  281.0712671149251
Iteration:  2003  ---> Frobenius norm:  281.07126449943416
Iteration:  2004  ---> Frobenius norm:  281.0712618862798
Iteration:  2005  ---> Frobenius norm:  281.07125927544
Iteration:  2006  ---> Frobenius norm:  281.0712566668929
Iteration:  2007  ---> Frobenius norm:  281.07125406061675
Iteration:  2008  ---> Frobenius norm:  281.0712514565903
Iteration:  2009  ---> Frobenius norm:  281.0712488547923
Iteration:  2

Iteration:  2134  ---> Frobenius norm:  281.07093594745265
Iteration:  2135  ---> Frobenius norm:  281.0709335121619
Iteration:  2136  ---> Frobenius norm:  281.07093107760153
Iteration:  2137  ---> Frobenius norm:  281.0709286437657
Iteration:  2138  ---> Frobenius norm:  281.07092621064857
Iteration:  2139  ---> Frobenius norm:  281.0709237782446
Iteration:  2140  ---> Frobenius norm:  281.0709213465482
Iteration:  2141  ---> Frobenius norm:  281.0709189155536
Iteration:  2142  ---> Frobenius norm:  281.07091648525545
Iteration:  2143  ---> Frobenius norm:  281.07091405564825
Iteration:  2144  ---> Frobenius norm:  281.0709116267266
Iteration:  2145  ---> Frobenius norm:  281.070909198485
Iteration:  2146  ---> Frobenius norm:  281.07090677091844
Iteration:  2147  ---> Frobenius norm:  281.0709043440214
Iteration:  2148  ---> Frobenius norm:  281.07090191778883
Iteration:  2149  ---> Frobenius norm:  281.0708994922155
Iteration:  2150  ---> Frobenius norm:  281.07089706729636
Iterati

KeyboardInterrupt: 

In [34]:
finalFilledMatrixReg = np.matmul(UReg, VReg.transpose())
finalFilledMatrixReg

array([[4.49799473, 4.38039355, 4.30098131, ..., 4.3575754 , 4.39683247,
        4.40638626],
       [4.0473849 , 3.92873628, 3.92665895, ..., 3.92097365, 3.96108725,
        3.9404496 ],
       [2.46744861, 2.42105336, 2.4730267 , ..., 2.45046193, 2.38492954,
        2.43660422],
       ...,
       [3.24143331, 3.04265777, 3.02061333, ..., 3.15216359, 3.23724569,
        3.14572616],
       [3.33905273, 3.27003492, 3.25786674, ..., 3.25421793, 3.27373297,
        3.26985059],
       [3.9475424 , 3.64592272, 3.59948687, ..., 3.62140673, 3.71903299,
        3.76569607]])

In [38]:
errorBatchReg = R-finalFilledMatrixReg
errorBatchReg[where_are_NaNs] = 0
np.linalg.norm(errorBatchReg)

281.1631788570524

## Biases included